In [45]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

In [46]:
c_to_f = lambda c :(9/5)*c + 32

In [47]:
#read in .xls to df
df=pd.read_excel("../n_obs/SIO_TEMP_20230501.xls",skiprows=47, header=None) #change excel filename
df = df.rename(columns={0: 'year',
                        1: 'month',
                        2: 'day',
                        5: 'sst'})
df = df[(df['year'] > 1916) & (df['year'] < 2023)] #exclude partial years 1916 & 2023
df['date'] = pd.to_datetime(df[['year','month','day']])
df

,year,month,day,3,4,sst,6,7,8,date
114,1917,1,1,NaN,NaN,NaN,0,NaN,NaN,1917-01-01
115,1917,1,2,NaN,NaN,11.5,0,NaN,NaN,1917-01-02
116,1917,1,3,NaN,NaN,11.9,0,NaN,NaN,1917-01-03
117,1917,1,4,NaN,NaN,11.5,0,NaN,NaN,1917-01-04
118,1917,1,5,NaN,NaN,11.7,0,NaN,NaN,1917-01-05
...,...,...,...,...,...,...,...,...,...,...
38825,2022,12,27,1440.0,0.0,14.7,0,14.6,0.0,2022-12-27
38826,2022,12,28,1320.0,0.0,15.1,0,14.9,0.0,2022-12-28
38827,2022,12,29,1430.0,0.0,14.9,0,14.9,0.0,2022-12-29
38828,2022,12,30,1719.0,0.0,15.3,0,15.1,0.0,2022-12-30


In [48]:
#make annual climatology and long-term mean
climatology = df.groupby(['year']).agg(['min','mean','max']).reset_index()
# long_term_mean = df.sst.mean()
# X = climatology.year.values.reshape(-1,1)
# Y = climatology[('sst','mean')].values.reshape(-1,1)
# lm = LinearRegression()
# lm.fit(X,Y)  # perform linear regression
# century_trend = (lm.coef_[0] * 100)[0]

text = []
#make hover text for chart
for i in range(len(climatology)):
    year = climatology.year[i]
    year_lo = climatology[('sst','min')][i]
    year_lo_f = c_to_f(year_lo)
    year_lo_day = datetime.strftime(df[(df.year == year) & (df.sst == year_lo)].date.iloc[0], '%b %-d')
    year_mean = climatology[('sst','mean')][i]
    year_mean_f = c_to_f(year_mean)
    year_hi = climatology[('sst','max')][i]
    year_hi_f = c_to_f(year_hi)
    year_hi_day = datetime.strftime(df[(df.year == year) & (df.sst == year_hi)].date.iloc[0], '%b %-d')
    #TODO bold year and/or bigger font
    #TODO make record high and low red and blue respectively
    text.append(f'<b>{year}</b><br>\
<span style="color: #67000d;">Warmest: {year_hi:.1f}°C / {year_hi_f:.1f}°F on {year_hi_day}</span><br>\
Average: {year_mean:.1f}°C / {year_mean_f:.1f}°F <br>\
<span style="color: #08306b;">Coldest: {year_lo:.1f}°C / {year_lo_f:.1f}°F on {year_lo_day}</span><br>')
text

['<b>1917</b><br><span style="color: #67000d;">Warmest: 23.1°C / 73.6°F on Jul 30</span><br>Average: 16.7°C / 62.1°F <br><span style="color: #08306b;">Coldest: 11.3°C / 52.3°F on Jan 16</span><br>',
 '<b>1918</b><br><span style="color: #67000d;">Warmest: 22.9°C / 73.2°F on Aug 2</span><br>Average: 17.4°C / 63.3°F <br><span style="color: #08306b;">Coldest: 12.5°C / 54.5°F on Feb 11</span><br>',
 '<b>1919</b><br><span style="color: #67000d;">Warmest: 22.8°C / 73.0°F on Jul 18</span><br>Average: 16.8°C / 62.3°F <br><span style="color: #08306b;">Coldest: 11.7°C / 53.1°F on Mar 15</span><br>',
 '<b>1920</b><br><span style="color: #67000d;">Warmest: 22.6°C / 72.7°F on Aug 10</span><br>Average: 16.6°C / 61.8°F <br><span style="color: #08306b;">Coldest: 12.0°C / 53.6°F on Feb 3</span><br>',
 '<b>1921</b><br><span style="color: #67000d;">Warmest: 23.3°C / 73.9°F on Jul 24</span><br>Average: 16.5°C / 61.8°F <br><span style="color: #08306b;">Coldest: 12.5°C / 54.5°F on Feb 15</span><br>',
 '<b>19

Using explicit colors as specified by Ed Hawkings

In [49]:
colors = ['#67000d', '#a50f15', '#cb181d', '#ef3b2c', '#fb6a4a', '#fc9272', '#fcbba1', '#fee0d2',
          '#deebf7', '#c6dbef', '#9ecae1', '#6baed6', '#4292c6', '#2171b5', '#08519c', '#08306b'][::-1]
#I copied these red to blue but colors needs to be blue to red

def get_color(sst):
    bin_step = (climatology[('sst','mean')].max() - climatology[('sst','mean')].min()) / len(colors)
    color_bins = np.arange(climatology[('sst','mean')].min(),climatology[('sst','mean')].max() + 0.001, bin_step) #stupid fix
    bin = np.min(np.argsort(np.abs(color_bins-sst))[0])
    if bin >= len(colors): return colors[len(colors) - 1] #TODO doublecheck logic
    else: return colors[bin]
get_color(19)

'#a50f15'

In [50]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
for i, sst in enumerate(climatology[('sst','mean')]):
    i += 1917
    fig.add_shape( #make the stripes
        type="rect",
        x0=i,
        y0=0,
        x1=i+1,
        y1=30,
        layer="below",
        line=dict(color="rgba(0,0,0,0)"),
        fillcolor=get_color(sst),
        secondary_y=False)
  
fig.add_trace(go.Scatter( #plot the annual mean
    x=df.groupby('year')['year'].first() + 0.5,
    y=climatology[('sst','mean')],
    name='Yearly average',
    mode='lines',
    line=dict(
        width=5,
        color='white'),
    text = text,
    hoverinfo='text',
    showlegend=False),
    secondary_y=False)

fig.add_trace(go.Scatter( #add an F axis
    x=df.groupby('year')['year'].first(),
    y=climatology[('sst','mean')].apply(c_to_f),
    name='Yearly average',
    mode='lines',
    line=dict(
        width=0,
        color='black'),
    hoverinfo='none',
    showlegend=False),
    secondary_y=True)

fig.update_layout(title_text='Interannual Sea Surface Temperature Trends at Scripps Pier',
    hovermode='x unified',
    plot_bgcolor='white')
fig.update_xaxes(dict(title='Year', range=[climatology.year.min(), climatology.year.max()], showgrid=False))
fig.update_yaxes(dict(title='Sea Surface Temperature (°C)', range=[climatology[('sst','mean')].min() - 0.5, climatology[('sst','mean')].max() + 0.5], showgrid=False, side='left'), secondary_y=False)
fig.update_yaxes(dict(title='Sea Surface Temperature (°F)', range=[c_to_f(climatology[('sst','mean')].min() - 0.5), c_to_f(climatology[('sst','mean')].max() + 0.5)], showgrid=False, side='right'), secondary_y=True)
pio.write_html(fig, 'interactive_showyourstripes_SIO.html') #TODO fix aspect ratio

In [51]:
fig